In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# 모든 컬럼 생략없이 출력하기 위한 옵션
pd.set_option('display.max_columns', None)

In [7]:
path = "../dataset/ml-latest-small/"
os.path.isdir(path)

True

# data load

In [10]:
rating_df = pd.read_csv(os.path.join(path + "ratings.csv"), encoding="utf-8")
movies_df = pd.read_csv(os.path.join(path + "movies.csv"), encoding="utf-8")
tags_df = pd.read_csv(os.path.join(path + "tags.csv"), encoding="utf-8")

# ▶ data 구성 확인

### 1. rating_df

In [11]:
rating_df.shape, rating_df.head()

((100836, 4),
    userId  movieId  rating  timestamp
 0       1        1     4.0  964982703
 1       1        3     4.0  964981247
 2       1        6     4.0  964982224
 3       1       47     5.0  964983815
 4       1       50     5.0  964982931)

### 2. movies_df

In [14]:
movies_df.shape, movies_df.head()

((9742, 3),
    movieId                               title  \
 0        1                    Toy Story (1995)   
 1        2                      Jumanji (1995)   
 2        3             Grumpier Old Men (1995)   
 3        4            Waiting to Exhale (1995)   
 4        5  Father of the Bride Part II (1995)   
 
                                         genres  
 0  Adventure|Animation|Children|Comedy|Fantasy  
 1                   Adventure|Children|Fantasy  
 2                               Comedy|Romance  
 3                         Comedy|Drama|Romance  
 4                                       Comedy  )

### 3. tags_df

In [13]:
tags_df.shape, tags_df.head()

((3683, 4),
    userId  movieId              tag   timestamp
 0       2    60756            funny  1445714994
 1       2    60756  Highly quotable  1445714996
 2       2    60756     will ferrell  1445714992
 3       2    89774     Boxing story  1445715207
 4       2    89774              MMA  1445715200)

# ▶ 평점 데이터 rating_df의 데이터 분석(기초 통계)

### ▷ 기본 정보 확인

In [25]:
rating_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


### ▷ 평점을 준 유저는 몇 명인가?

In [20]:
len(rating_df.groupby(["userId"]))

610

In [21]:
# unique()로 구할 수도 있음
len(rating_df.userId.unique())

610

### ▷ 평점을 받은 영화의 수는 몇 개인가? 

In [22]:
len(rating_df.movieId.unique())  # 9742개의 영화 중 평점 받은 영화 9742개

9724

### ▷ 평점의 평균, 표준편차 등 통계에 대한 데이터를 확인

In [24]:
rating_df.rating.describe()
# 평균이 3.5이고 표준편차가 1.0 정도이므로 점수가 2.5~4.5 사이에 많이 분포되어 있음을 알 수 있음

count    100836.000000
mean          3.501557
std           1.042529
min           0.500000
25%           3.000000
50%           3.500000
75%           4.000000
max           5.000000
Name: rating, dtype: float64

### ▷ NaN 유무 확인 : ```isnull()```

In [26]:
rating_df.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

# ▶ ```groupby()```를 활용한 통계 연습

### ▷ 유저별 평점 평균
userId가 1인 사람의 평균 평점, 2인 사람의 평균 평점, ...

In [31]:
rating_df.groupby('userId').mean()['rating']
# == rating_df.groupby('userId')['rating'].mean()

userId
1      4.366379
2      3.948276
3      2.435897
4      3.555556
5      3.636364
         ...   
606    3.657399
607    3.786096
608    3.134176
609    3.270270
610    3.688556
Name: rating, Length: 610, dtype: float64

### ▷ 유저별 평점별 평가 횟수
1번 유저가 1점을 몇 번 줬는지, 2점을 몇 번 줬는지 등에 대한 통계

In [32]:
rating_df.groupby(['userId', 'rating']).size()

userId  rating
1       1.0         1
        2.0         5
        3.0        26
        4.0        76
        5.0       124
                 ... 
610     3.0       230
        3.5       315
        4.0       286
        4.5       148
        5.0       180
Length: 3976, dtype: int64

### ▷ 유저가 평점을 준 영화의 수

In [38]:
rating_df.groupby("userId")["movieId"].count()

userId
1       232
2        29
3        39
4       216
5        44
       ... 
606    1115
607     187
608     831
609      37
610    1302
Name: movieId, Length: 610, dtype: int64

### ▷ 유저가 평균적으로 준 평점과 평점을 준 영화의 수 -> DataFrame으로 반환

In [53]:
user_avg_rating = rating_df.groupby('userId')['rating'].mean()  # 유저별 평균 평점
user_rating_movie_cnt = rating_df.groupby("userId")["movieId"].count()  # 유저별 평점 영화 수

stat_df = pd.concat([user_avg_rating, user_rating_movie_cnt], axis=1)
stat_df

,rating,movieId
userId,,
1,4.366379,232
2,3.948276,29
3,2.435897,39
4,3.555556,216
5,3.636364,44
...,...,...
606,3.657399,1115
607,3.786096,187
608,3.134176,831


In [55]:
stat_df2 = pd.DataFrame({
    "rating_mean": user_avg_rating,
    "movie_count": user_rating_movie_cnt
    })
stat_df2

,rating_mean,movie_count
userId,,
1,4.366379,232
2,3.948276,29
3,2.435897,39
4,3.555556,216
5,3.636364,44
...,...,...
606,3.657399,1115
607,3.786096,187
608,3.134176,831
